In [35]:
import myComponents as tc
import topic as tp
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import random

In [50]:
# network parameter
num_broker = 10000
num_sub = 5000
num_pub = 4000
broker_r = 500
broker_rates = [broker_r]*num_broker
topic_dist = [10,5,5,5]
monitor_rate = 1
runtime = 100
connection_style = None

# sub parameter
avg_sub_size = 10.0
sub_r = 3

sub_rates = [sub_r]*num_sub
sub_num_topic = [10]*num_sub
sub_diameter = [len(topic_dist)*2]*num_sub

# pub parameter
avg_pub_size = 20.0
pub_r = 3

pub_rates = [pub_r]*num_pub
pub_num_topic = [10]*num_pub
pub_diameter = [len(topic_dist)]*num_pub


seed = 1

In [44]:
tc.SwitchPort.mode = 'PF'
total_topic = tp.TopicTree()
total_topic.random_construct(topic_dist, seed)

In [45]:
net = tc.Network(total_topic, avg_sub_size, avg_pub_size)
net.initialize_nodes(broker_rates, sub_rates, sub_num_topic, sub_diameter, pub_rates, pub_num_topic, pub_diameter, monitor_rate, seed)


In [49]:
for sub in net.sub_list:
    if len(sub.topic_list)!=10:
        print(len(sub.topic_list))

In [51]:
tc.SwitchPort.mode = 'PF'
total_topic = tp.TopicTree()
total_topic.random_construct(topic_dist, seed)
#total_topic.visualize(total_topic.root)
net = tc.Network(total_topic, avg_sub_size, avg_pub_size)
net.initialize_nodes(broker_rates, sub_rates, sub_num_topic, sub_diameter, pub_rates, pub_num_topic, pub_diameter, monitor_rate, seed)
net.establish_topology(seed)
net.connect_client(connection_style, seed)
net.env.run(runtime)
print(tc.SwitchPort.mode)

KeyboardInterrupt: 

In [ ]:
net.visualize()
plt.figure(figsize=(10, 3), dpi=80)
#monitor.queue_size
#monitor.queue_length
for i in range(len(net.broker_monitor_list)):
    monitor = net.broker_monitor_list[i]
    plt.plot(monitor.queue_length, label=monitor.port.sp_id)
plt.legend(bbox_to_anchor=(1.01, 1),loc='upper left')
plt.title("queue length")
window = 6
plt.figure(figsize=(10, 3), dpi=80)
for i in range(len(net.broker_monitor_list)):
    monitor = net.broker_monitor_list[i]
    a = np.array(monitor.bytes_sent)
    b = np.array([0]*window+monitor.bytes_sent[0:-window])
    plt.plot((a-b)[window-1:]/window/monitor_rate, label=monitor.port.sp_id)
plt.legend(bbox_to_anchor=(1.01, 1),loc='upper left')
plt.title("output rate in byte/s, window="+str(window))
import warnings
warnings.filterwarnings('ignore')
window = 6
plt.figure(figsize=(10, 9), dpi=80)

for i in range(len(net.sub_monitor_list)):
    monitor = net.sub_monitor_list[i]
    a = np.array(monitor.packets_rec)
    b = np.array([0]*window+monitor.packets_rec[0:-window])
    avg_packet_rec_per_time = (a-b)[window-1:]/window/monitor_rate
    
    a = np.array(monitor.tot_waits)
    b = np.array([0]*window+monitor.tot_waits[0:-window])
    avg_waits_per_time = (a-b)[window-1:]/window/monitor_rate
    

    plt.subplot(311)
    plt.plot(avg_waits_per_time, label=monitor.client.client_id)
    plt.subplot(312)
    plt.plot(avg_packet_rec_per_time, label=monitor.client.client_id)
    plt.subplot(313)
    plt.plot(avg_waits_per_time/avg_packet_rec_per_time, label=monitor.client.client_id)

plt.subplot(311)
plt.title("avg_waits s/s, window="+str(window))
plt.subplot(312)
plt.title("avg_pkt_recieved /s, window="+str(window))
plt.subplot(313)
plt.title("avg_waits/pkt, window="+str(window))
plt.legend(bbox_to_anchor=(1.01, 1),loc='upper left')
plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.25,wspace=0.35)